#Aula 1 Versão para VSC

In [2]:
%pip install -qU -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Importação da API Key com mudancas para funcionar no vsc

In [ ]:

import os, google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

c:\Users\sagui\OneDrive\Documentos\MeusProjetosGit\Imersao_Agentes_IA_Gemini\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


"\nfrom google.colab import userdata\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nGOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')\n"

Conexão com o Gemini

In [4]:
from pydantic import SecretStr

lln = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0,
    api_key=SecretStr(GOOGLE_API_KEY)
)

Teste básico do modelo com uma pergunta

In [5]:
resp_teste = lln.invoke('Quem é voce?')
print(resp_teste.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [6]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
)

Definição do Modelo de Saída com Pydantic

In [7]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal['AUTO_RESOLVER', 'PEDIR_INFO', 'ABRIR_CHAMADO']
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

Criação do LLM para Classificação e da Chain

In [8]:
lln_triagem = ChatGoogleGenerativeAI (
    model='gemini-2.5-flash',
    temperature=0,
    api_key=SecretStr(GOOGLE_API_KEY)
)

Importa mensagens e cria a chain com saída estruturada.

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = lln_triagem.with_structured_output(TriagemOut)

def triagem(mensagem : str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()

Testes do Agente

In [10]:
testes =  ["Posso reembolsar a internet?",
           "Quero ter mais 5 dias remoto, Como faco?",
           "Posso reembolsar cursos ou treinamentos do Alura",
           "Quantas capivaras tem no Rio Pinheiros?"]

for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero ter mais 5 dias remoto, Como faco?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: Posso reembolsar cursos ou treinamentos do Alura
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantas capivaras tem no Rio Pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['informação sobre políticas internas da empresa']}

